# Implementación de la regresión logística con el método de la máxima verosimilitud

## Definir la función de entorno $L(\beta)$

Recordamos que ya no hay $\alpha$, sino $\beta_0$

$$L(\alpha,\beta) = \prod^n_{i=1} P_i^{y_i} \cdot (1 - P_i)^{1 - y_i}$$<br>

In [123]:
def likelihood(y, p):
    totalprod = 1
    for i in range(len(y)):
        row = np.where(y[i] == 1, p[i], 1 - p[i])
        totalprod = totalprod * row
    return totalprod

## Calcular las probabilidades para cada observación

Las probabilidades se calculan con la función sigmoide:

$$P(x_i) = \frac{1}{1 + e^{-\beta x_i}}$$

In [54]:
def probs(beta, X):
    import numpy as np
    return 1/(1 + np.exp(-np.transpose(np.matrix(beta))*np.transpose(np.matrix(X))))
#en teoría devuelve las probabilidades en 1xn
#seguramente no funcione y haya que pasar beta y X a matrices o algo así... o usar np.prod() para hacer el prod matricial...

## Calcular la matriz diagonal W

$$ W = diag(P_i \cdot (1 - P_i))^n_{i=1} $$

In [81]:
def diagw(p):
    import numpy as np
    w = np.zeros(len(p) * len(p)).reshape(len(p), len(p))

    for i in range(len(p)):
        w[i,i] = (p[i] * np.transpose(np.matrix(1 - p[i]))).astype(float)
    return w

## Definir la función logística

E implementar también el método de Newton-Raphson.<br>
$$ \beta_{n+1} = \beta_n - \frac{f(\beta_n)}{f'(\beta_n)} $$

$$f(\beta) = X(Y - P)$$
$$f'(\beta) = X\omega X^t$$

In [79]:
def logistic(X_original, y, limit):
    import numpy as np
    from numpy import linalg
    
    nrows = np.shape(X_original)[0]
    ncols = np.shape(X_original)[1] + 1
    
    bias = np.ones(nrows).reshape(nrows, 1)
    X = np.append(bias, X_original, axis = 1)

    #Inicializamos las betas a 0
    beta = np.zeros(ncols).reshape(ncols, 1)
    #En este array iremos guardando las diferencias entre cada iteración para comprobar cuando parar
    beta_dif = np.array(range(ncols)).reshape(ncols, 1)
    
    #Paramos cuando el cuadrado de las diferencias sea menor que el limite establecido
    while(sum(beta_dif * beta_dif) > limit):
        
        p = np.transpose(np.matrix(probs(beta, X)))
        w = diagw(p)

        f_beta = np.transpose(np.matrix(X)) * (y - p)
        derivate_f_beta = np.transpose(np.matrix(X)) * w * X

        beta_dif = np.array(linalg.inv(derivate_f_beta) * f_beta)
        beta = beta + beta_dif
        
    return beta

### Probando nuestra implementación

In [127]:
import numpy as np

nrows = 10

X = np.array(range(nrows)).reshape(nrows, 1)
y = np.array([0, 0, 0, 0, 1, 0, 1, 0, 1, 1]).reshape(nrows, 1)

beta = logistic(X, y, 0.00001)

beta

array([[-3.69557172],
       [ 0.66220827]])

In [129]:
#Esto no entiendo muy bien que implica, la función de coste?? Nos sale diferente a él y a mi
#Sin embargo las betas me salen iguales
likelihood(y, probs(beta, np.append(np.ones(nrows).reshape(nrows, 1), X, axis=1)).reshape(nrows, 1))

array([[ 0.01343191]])

### Probando la diferencia con el paquete de Python

In [138]:
import statsmodels.api as sm

#Para llamar a la regresión logística de statsmodels hay que añadir antes el bias
model = sm.Logit(y, np.append(np.ones(nrows).reshape(nrows, 1), X, axis=1))
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [143]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        8
Method:                           MLE   Df Model:                            1
Date:                Fri, 15 Jun 2018   Pseudo R-squ.:                  0.3596
Time:                        17:57:50   Log-Likelihood:                -4.3101
converged:                       True   LL-Null:                       -6.7301
                                        LLR p-value:                   0.02781
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -3.6956      2.289     -1.615      0.106        -8.182     0.791
x1             0.6622      0.400      1.655      0.098        -0.122     1.446
==============================================================================
"""